In [3]:
import urllib.request, json
import folium
import urllib.request, json
from datetime import datetime as dt
import time

import pandas as pd

def return_json(url_api):
    with urllib.request.urlopen(url_api) as url:
        data = json.load(url)
        return data
def weather_api_coordinates(lat, long):
    url_api = "https://api.weather.gov/points/" + str(lat) + ',' + str(long)
    return(return_json(url_api))

def polygon_to_map(coordinates): #coordinates in the format of [[lat, lon]...]
    m = folium.Map(location=coordinates[0], zoom_start=13)
    folium.Polygon(
        locations = coordinates,  # Flip lon/lat to lat/lon
        color='blue',
        fill=True,
        fill_color='cyan',
        fill_opacity=0.5
    ).add_to(m)
    return(m)

api_weather_url_base = "https://api.weather.gov/points/"
api_weather_url = api_weather_url_base + "47.675908,-122.208008"
api_weather_url = api_weather_url_base + "46.189070,-122.184252"
start = time.time()
data = return_json(api_weather_url)
print(time.time()-start)
forecast = return_json(data['properties']['forecast'])
print(time.time()-start)

print(api_weather_url)

print(data['properties']['forecast'])
print(data['properties']['forecastHourly'])
print(data['properties']['forecastGridData'])
print(data['properties']['observationStations'])

0.7377879619598389
1.1124556064605713
https://api.weather.gov/points/46.189070,-122.184252
https://api.weather.gov/gridpoints/PQR/135,131/forecast
https://api.weather.gov/gridpoints/PQR/135,131/forecast/hourly
https://api.weather.gov/gridpoints/PQR/135,131
https://api.weather.gov/gridpoints/PQR/135,131/stations


In [4]:
temp = return_json(data['properties']['forecastGridData'])

In [7]:
forecastGrid = temp.copy()

In [13]:
for k in forecastGrid['properties'].keys():
    print(k)


@id
@type
updateTime
validTimes
elevation
forecastOffice
gridId
gridX
gridY
temperature
dewpoint
maxTemperature
minTemperature
relativeHumidity
apparentTemperature
wetBulbGlobeTemperature
heatIndex
windChill
skyCover
windDirection
windSpeed
windGust
weather
hazards
probabilityOfPrecipitation
quantitativePrecipitation
iceAccumulation
snowfallAmount
snowLevel
ceilingHeight
visibility
transportWindSpeed
transportWindDirection
mixingHeight
hainesIndex
lightningActivityLevel
twentyFootWindSpeed
twentyFootWindDirection
waveHeight
wavePeriod
waveDirection
primarySwellHeight
primarySwellDirection
secondarySwellHeight
secondarySwellDirection
wavePeriod2
windWaveHeight
dispersionIndex
pressure
probabilityOfTropicalStormWinds
probabilityOfHurricaneWinds
potentialOf15mphWinds
potentialOf25mphWinds
potentialOf35mphWinds
potentialOf45mphWinds
potentialOf20mphWindGusts
potentialOf30mphWindGusts
potentialOf40mphWindGusts
potentialOf50mphWindGusts
potentialOf60mphWindGusts
grasslandFireDangerIndex
prob

In [24]:
data['properties']['forecastHourly']
temp = return_json(data['properties']['forecastHourly'])

In [32]:
forecastHourly = temp.copy()
del forecastHourly['@context']
del forecastHourly['type']
del forecastHourly['geometry']
del forecastHourly['properties']['elevation']

forecastHourly['properties']

{'units': 'us',
 'forecastGenerator': 'HourlyForecastGenerator',
 'generatedAt': '2024-12-10T04:27:56+00:00',
 'updateTime': '2024-12-10T04:13:24+00:00',
 'validTimes': '2024-12-09T22:00:00+00:00/P7DT9H',
 'periods': [{'number': 1,
   'name': '',
   'startTime': '2024-12-09T20:00:00-08:00',
   'endTime': '2024-12-09T21:00:00-08:00',
   'isDaytime': False,
   'temperature': 28,
   'temperatureUnit': 'F',
   'temperatureTrend': '',
   'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 0},
   'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': -15},
   'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 36},
   'windSpeed': '12 mph',
   'windDirection': 'NNW',
   'icon': 'https://api.weather.gov/icons/land/night/few?size=small',
   'shortForecast': 'Mostly Clear',
   'detailedForecast': ''},
  {'number': 2,
   'name': '',
   'startTime': '2024-12-09T21:00:00-08:00',
   'endTime': '2024-12-09T22:00:00-08:00',
   'isDaytime': False,
   'temperature': 28,
   'tempe

In [8]:
df = pd.DataFrame(forecast['properties']['periods'])
df['startTime'] = pd.to_datetime(df['startTime'])
df['startTime'] = df['startTime'].apply(lambda x : x.strftime('%Y-%m-%d %I:%M:%S %p'))
#df = df.set_index('startTime')
df['windSpeed_min'] = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[0]))
df['windSpeed_max']  = df['windSpeed'].apply(lambda x: int(x.replace(' mph', '').split(' to ')[-1]))
df.drop(['number', 'endTime', 'temperatureTrend', 'windSpeed'], axis = 1, inplace = True)
df['probabilityOfPrecipitation'] = df['probabilityOfPrecipitation'].apply( lambda x: x['value'] / 100 if x['value'] is not None else 0 )

df.head()

,name,startTime,isDaytime,temperature,temperatureUnit,probabilityOfPrecipitation,windDirection,icon,shortForecast,detailedForecast,windSpeed_min,windSpeed_max
0,Tonight,2024-12-09 08:00:00 PM,False,26,F,0.0,NNE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 26. North nort...",12,18
1,Tuesday,2024-12-10 06:00:00 AM,True,38,F,0.0,SE,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny. High near 38, with temperatures ...",9,17
2,Tuesday Night,2024-12-10 06:00:00 PM,False,30,F,0.0,S,https://api.weather.gov/icons/land/night/bkn?s...,Mostly Cloudy,"Mostly cloudy, with a low around 30. South win...",12,16
3,Wednesday,2024-12-11 06:00:00 AM,True,33,F,0.3,SSE,https://api.weather.gov/icons/land/day/bkn/sno...,Mostly Cloudy then Chance Light Snow,"A chance of snow after 4pm. Mostly cloudy, wit...",8,14
4,Wednesday Night,2024-12-11 06:00:00 PM,False,25,F,0.6,SE,"https://api.weather.gov/icons/land/night/snow,...",Snow Likely,"Snow likely. Mostly cloudy, with a low around ...",8,17


In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot temperature on the primary y-axis
ax1.plot(df['startTime'], df['temperature'], color='tab:red', label='Temperature (°F)')
ax1.set_xlabel('Date Time')
ax1.set_ylabel('Temperature (°F)', color='tab:red')
min_temperature = df['temperature'].min() - 5  # adding buffer
max_temperature = df['temperature'].max() + 5  # adding buffer
ax1.set_ylim(min_temperature, max_temperature)
ax1.tick_params(axis='y', labelcolor='tab:red')

# Create a second y-axis to plot probability of precipitation
ax2 = ax1.twinx()
ax2.plot(df['startTime'], df['probabilityOfPrecipitation'], color='tab:blue', label='Probability of Precipitation', linestyle='--')
ax2.set_ylabel('Probability of Precipitation', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim((0,1))

# Title and show the plot
plt.title('Temperature and Probability of Precipitation')
fig.tight_layout()
plt.show()


In [ ]:
dt.fromisoformat(forecast['properties']['periods'][0]['startTime'])


In [ ]:
from datetime import datetime as dt
dt.fromisoformat(forecast['properties']['generatedAt'])

In [ ]:
data['properties']

In [ ]:
forecastHourly = return_json(data['properties']['forecastHourly'])
forecastHourly['properties']